In [ ]:
import aisuite as ai

client = ai.Client()

# Define the local Gemma model
model_name = "ollama:gemma3" 

messages = [{"role": "user", "content": "Explain quantum physics in one sentence."}]

response = client.chat.completions.create(
    model=model_name,
    messages=messages
)

print(response.choices[0].message.content)

In [ ]:
import ollama
import datetime

# Define your tool
def get_current_time():
    return str(datetime.datetime.now())

# Define the tool for the model, this didn't work with current gemma3
tools = [{
    'type': 'function',
    'function': {
        'name': 'get_current_time',
        'description': 'Get the current time',
    },
}]


system_prompt = """
You have access to the following tool:
{ "name": "get_current_time", "description": "Returns current time" }

If the user asks for the time, respond ONLY with:
CALL: get_current_time()
"""


messages=[
        {'role': 'system', 'content': system_prompt},
        {'role': 'user', 'content': 'What time is it?'}
]


response = client.chat.completions.create(
    model="ollama:gemma3",
    messages=messages,
    max_turns=5
)
print(response.choices[0].message.content)

content = response.choices[0].message.content
if "CALL: get_current_time()" in content:
    # Step 2: Run the actual Python code
    time_result = get_current_time()
    print(f"--- Executing Tool: get_current_time() -> {time_result} ---")

    # Step 3: Feed the result back to the model
    messages.append({'role': 'assistant', 'content': content})
    messages.append({'role': 'user', 'content': f"Tool result: {time_result}"})
    
    final_response = ollama.chat(model='gemma3', messages=messages)
    print("Gemma says:", final_response['message']['content'])
else:
    print("Gemma says:", content)

In [ ]:
import ollama
import datetime

# Define your tool
def get_current_time():
    return str(datetime.datetime.now())

# Define the tool for the model, this didn't work with current gemma3
tools = [{
    'type': 'function',
    'function': {
        'name': 'get_current_time',
        'description': 'Get the current time',
        "parameters": {}
    },
}]

# Message structure
prompt = "What time is it?"
messages = [
    {
        "role": "user",
        "content": prompt,
    }
]

# this can be fixed with workaround
response = client.chat.completions.create(
    model="ollama:gemma3",
    messages=messages
    # tools=[get_current_time]
   
)
print(response.choices[0].message.content)


## recommended workaround

# FROM gemma3
# # This template tells Ollama how to inject tools into the prompt
# TEMPLATE """{{ if .System }}<|im_start|>system
# {{ .System }}<|im_end|>
# {{ end }}{{ if .Prompt }}<|im_start|>user
# {{ .Prompt }}<|im_end|>
# {{ end }}<|im_start|>assistant
# {{ if .Tools }}<|im_start|>tools
# {{ .Tools }}<|im_end|>
# {{ end }}{{ .Response }}<|im_end|>"""


# ollama create gemma3-tools -f Modelfile